In [99]:

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select

#installation guide for selenium
#https://medium.com/@shanyitan/how-to-install-selenium-and-run-it-successfully-via-jupyter-lab-c3f50d22a0d4

In [100]:
#gathering information function
def gather(browser):
    lst=[]
    try:
        x=1
        while True: 
            x_pather = "//*[@id='myBody']/table/tbody/tr/td/table/tbody/tr[5]/td/form/table/tbody/tr[3]/td/a["+str(x)+"]"
            main = browser.find_element_by_xpath(x_pather)
            x+=1
            lst.append(main)
    except:
        lst.pop()
    return lst

### Scraping the data with selenium
*TODO we need to make it run by page

*TODO testing with other maslulim (Economics and statistics is good but computers not)

In [231]:
browser = webdriver.Chrome()
browser.get("https://shnaton.huji.ac.il/")
scraped_data = []
hug_by_fac = {'01': ['0255', '0176', '0150', '0195', '0145', '0158', '0114', '0118', '0249', '0181', '0192', '0151', '0105', '0142', '0109', '0222', '0200', '0238', '0241', '0240', '0242', '0243', '0231', '0237', '0232', '0239', '0112', '0246', '0124', '0122', '0201', '0101', '0160', '0230', '0155', '0172', '0179', '0156', '0141', '0311', '0143', '0298', '0199', '0113', '0301', '0180', '0108', '0117', '0140', '0300', '0244', '0699', '0157', '0123', '0197', '0198', '0202', '0144', '0115', '0102', '1767', '1506', '1811', '1511', '1057', '1223', '1515', '1411', '1561', '1790', '2990', '1136', '1802', '1084', '1401', '1971', '1248', '1016', '1027', '2995', '2885', '1069'],
                 '02': ['0592', '0555', '0579', '0809', '0890', '0577', '0573', '0802', '0821', '0590', '0575', '0501', '0529', '0589', '0581', '0588', '0880', '0560', '0321', '0101', '0553', '0545', '0570', '0566', '0576', '0521', '0591', '0572', '0143', '0595', '0530', '0531', '0596', '0320', '0824', '0543', '0541', '0511', '0318', '0502', '0611', '0569', '0520', '5881', '5882', '5602', '5954', '5701', '5905', '5221', '5601', '5222', '5920', '5210', '5952', '5311', '5422', '5212', '5998'], 
                 '03': ['0416', '0222', '0405', '0414', '0401', '0411'], 
                 '04': ['0698', '0603', '0616', '0624', '0602', '0602', '0626', '0618', '0608', '0606', '0610', '0612', '0619', '0630', '0621', '0621', '0640', '0625', '0607', '0627', '0627', '0601', '0681', '0699', '0613'],
                 '05': ['0617', '0611'], 
                 '06': ['0370', '0325', '0343', '0322', '3242', '3227', '1008', '3224', '1006', '3221', '3223', '3226', '1002', '3220', '3245', '3234', '3228', '1004', '3243', '3244'],
                 '07': ['0802', '0810', '0821', '0358', '0350', '0312', '0321', '0337', '0364', '0365', '0369', '0335', '0330', '0311', '0816', '0815', '0301', '0320', '0824', '0300', '0318', '0363', '0326', '0323', '3020', '3651', '8030', '8024', '8032', '8022', '8020', '3119', '3017', '3110', '5903'],
                 '08': ['0715', '0792', '0728', '0890', '0722', '0793', '0723', '0735', '0737', '0713', '0731', '0721', '0732', '0799', '0795', '0738', '0717', '0718', '0794', '0710', '0791', '0716', '0712', '0733', '0714', '0729', '0730', '7725', '7726'],
                 '09': ['0434', '0433', '0431', '0432'], 
                 '11': ['0597', '5970'], 
                 '12': ['0525', '0523', '0583', '0587', '0586', '0521', '0532', '0591', '0595', '0322', '0530', '0541', '0520', '0323', '5210', '5311', '5422', '5212'], 
                 '16': ['0855'],
                 '30': ['0728', '0809', '0815', '0502', '8997']}


lst_toar = [0,1,2,3,4,5,6,7,8]

hugim_checked = []
faculty_checked = []
maslul_checked = []

for faculty in hug_by_fac.keys():
    faculty_checked.append(faculty)
    try:
        sleep(1)
        select = Select(browser.find_element_by_id('faculty'))
        select.select_by_value(str(faculty)) #Pass value as string
        sleep(1)

        for hug in hug_by_fac[faculty]: 
            hugim_checked.append(hug)
            sleep(1.2)
            select = Select(browser.find_element_by_id('faculty'))
            select.select_by_value(str(faculty)) #Pass value as string
            sleep(1.2)
            select = Select(browser.find_element_by_id('hugim'))
            select.select_by_value(str(hug))
            sleep(1.2)

            maslul_options = browser.find_elements(By.XPATH,'//*[@name="maslul"]/option')
            #second loop over maslul
            maslul_lst = []

            for mas in maslul_options:
                if mas.get_attribute("value") == "0" or mas.text == "בחרו חוג":
                    pass
                else:
                    maslul_lst.append(mas.get_attribute("value")) 

            for maslul in maslul_lst:
                try:
                    if maslul is not None:
                        maslul_checked.append(maslul)
                    sleep(1.5)
                    select = Select(browser.find_element_by_id('faculty'))
                    select.select_by_value(str(faculty)) #Pass value as string
                    sleep(1.5)

                    select = Select(browser.find_element_by_id('hugim'))
                    select.select_by_value(str(hug))
                    sleep(1.5)

                    select = Select(browser.find_element_by_id('maslul'))
                    select.select_by_value(str(maslul))
                    sleep(1.5)

                    for toar in lst_toar:
                        try:
                            select = Select(browser.find_element_by_id('toar'))
                            select.select_by_value(str(toar))
                            sleep(1.2)

                            select = Select(browser.find_element_by_id('shana'))
                            select.select_by_value('0')
                            sleep(1.2)

                            browser.find_element_by_css_selector("input[type='radio'][id='full']").click()
                            sleep(1.2)

                            browser.find_element_by_css_selector("input[type='button'][value=' חפש ']").click()
                        except:continue
                        #collecting information
                        try:
                            while True:#information gathering function here
                                browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                                sleep(1)
                                for i in gather(browser):
                                    scraped_data.append(i.text)
                                browser.find_element_by_xpath("//a[@title='העמוד הבא']").click()
                        except:
                            browser.get("https://shnaton.huji.ac.il/")
                except:
                    browser.get("https://shnaton.huji.ac.il/")

    except:
        browser.close()
        print("error in faculty:"+str(faculty)+"hug:"+str(hug)+"maslul:"+str(maslul)+"toar:"+str(toar))
browser.close()

In [ ]:
#getting the failing scraper checklist
print("error in faculty: ",str(faculty),"\n error in hug: ",str(hug),"\n error in maslul: ",str(maslul),"\n")
print("faculty checked:",sorted(faculty_checked),"\n hugim checked:",sorted(hugim_checked),"\n maslul checked:",sorted(maslul_checked))

### Gathering the data from the scrpaed text
gathering the data to dictionaries with regex.
we might have some weak points here.
#TODO test with other maslulim.

#### infrastructure 
database = {id:["semester": א\ב\שנתי, "day": א\ב\ג\ד\ה\ו, "hours": "hh:mm-hh:mm",....,..]}

from outside to inside:
big dictionary keys are for course_id, the value is a list which every item in the list is a different hour of the class.

In [232]:
#manualy continue the scraper
scraped_data_copy2 = scraped_data.copy()


In [230]:
#creating a copy of scraped_data
scraped_data_copy =scraped_data.copy()


In [233]:
scraped_data= scraped_data_copy+scraped_data_copy2

##### regex

In [240]:
def data_gatherer(scraped_data):
    """ Return (dictionary that contains all the courses per page, canceled courses)"""
    database = dict()
    canceled_courses = []
    for course in scraped_data:
        course_details = []
        info_by_day = dict()
        try:
            name=re.findall(r"(\n[a-z,A-Z]\D{2,}\|)", course)[0].split("|")
            name = re.sub(r"[\n\t]*", "", name[0]).replace("  "," ")

        except:
            name = 'no name'
            
        hours = (re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", course))
        course_id = (re.findall(r"\|\s{3}(\d{5})", course))
        day = (re.findall(r"\n(יום [אבגדהו])", course))
        semester = (re.findall(r"\n(שנתי|סמסטר [אבגדהו])", course))
          
        #checking for canceled courses
        if len(course_id) == 0 or len(hours) == 0:
            if len(hours) == 0:
                canceled_courses.append([course_id,name])
            pass

        else:
            for item in range(len(semester)):
                info_by_day={}
                info_by_day["semester"] = semester[item] 
                try:
                    info_by_day["day"] = day[item]
                except:
                    info_by_day["day"] = 'None'
                try:
                    info_by_day["hours"] = hours[item]
                except:
                    info_by_day["hours"] = 'None' 
                course_details.append(info_by_day)
            database[course_id[0]] = {name:course_details}

    canceled_courses = [x for x in canceled_courses if x != []]
    return database, canceled_courses
database, canceled_courses = data_gatherer(scraped_data)

#### cleaning and sorting data

In [243]:

database_sorted = {}
for key, value in sorted(database.items()): # Note the () after items!
    database_sorted[key] = value

#removing duplicates and sorting removed
dicto = {}
for course in canceled_courses:
    dicto[str(course)] = None
canceled_sorted=sorted((dicto))
print("\n\n number of active courses in updated scraper:",len(database_sorted.keys()))
print("\n\n list of "+str(len(canceled_sorted)))




 number of active courses in updated scraper: 4712


 list of 1309


###### saving a part of the list as text file

In [247]:
text_sample= scraped_data[0:30]
import json
with open('text_sample.txt', 'w') as f:
    f.write(json.dumps(text_sample))

#### To do list:
* scraper still getting stuck after a while (due to website maintenance) - maybe using try;except???

* saving scraper checkpoints in case of error so we could start from the same checkpoint.

note: duplication with courses does not excist due to using a dictionary

* adding to each course the group (א,ב,ג,ד....)
* adding to each course if מעבדה/תרגול/שיעור/מבחן....

* finally creating csv file with all the courses information